In [1]:
import pandas as pd
import glob
import os
from datetime import datetime

In [2]:
# Constants
CURRENT_YEAR = datetime.now().year
CURRENT_MONTH = datetime.now().month
folder_path = "../Game/State_Policy_Data/State_Policies" # Pull the NFIP Folder from Cloud
constant_discount = False

# Gather all file paths
csv_files = glob.glob(os.path.join(folder_path, "propertyState_*.csv"))
csv_files += glob.glob(os.path.join(folder_path, "FL", "FL_countyCode_*.csv"))
csv_files += glob.glob(os.path.join(folder_path, "LA", "LA_countyCode_*.csv"))

In [3]:
# Helper function to calculate present value
def calculate_present_value(amount, policy_year):
    years_diff = CURRENT_YEAR - policy_year
    return amount / ((1 + INTEREST_RATE) ** years_diff)

# Helper function to compound past values to today's value
def compound_to_present(amount, years_diff, months_diff, rate):
    time_diff = years_diff + (months_diff / 12)  # Convert month difference to fraction of a year
    return amount * ((1 + rate / 100) ** time_diff)

In [4]:
if constant_discount:
    # Constants
    INTEREST_RATE = 3.75 / 100
    
    # Dictionary to hold annualized sum by year
    annualized_premiums = {}
    
    # Process each CSV file
    for file in csv_files:
        # Print the file name as it’s being processed
        print(f"Processing file: {file}")
        df = pd.read_csv(file, parse_dates=["policyEffectiveDate"])
        
        # Calculate the present value of premiums for each record
        df["policyYear"] = df["policyEffectiveDate"].dt.year
        df["adjustedPremium"] = df.apply(lambda row: calculate_present_value(row["totalInsurancePremiumOfThePolicy"], row["policyYear"]), axis=1)
        
        # Sum adjusted premiums by year
        for year, yearly_sum in df.groupby("policyYear")["adjustedPremium"].sum().items():
            if year not in annualized_premiums:
                annualized_premiums[year] = 0
            annualized_premiums[year] += yearly_sum
    
    # Calculate the total annualized sum
    total_annualized_sum = sum(annualized_premiums.values())
    
    # Print the final annualized sum
    print(f"Total annualized sum of adjusted insurance premiums: {total_annualized_sum/(2023-1979+1):.2f}")

In [5]:
# Load and preprocess discount rate data
discount_rate_file = "BLS_Data/FRED_discount_Monthly.csv"  # Replace with the correct path
discount_rates = pd.read_csv(discount_rate_file, parse_dates=["DATE"])
discount_rates["month"] = discount_rates["DATE"].dt.month
discount_rates["year"] = discount_rates["DATE"].dt.year
discount_rates = discount_rates.rename(columns={"INTDSRUSM193N": "discountRate"})

In [ ]:
#CPI Inflation Adjust
curr_rate = 313.3 # Update end of year

In [6]:
# Running total for compounded sum
total_compounded_sum = 0

# Process each file
for file in csv_files:
    print(f"Processing file: {file}")
    
    # Load the file and extract required columns
    df = pd.read_csv(file, parse_dates=["policyEffectiveDate"], usecols=["policyEffectiveDate", "federalPolicyFee", "totalInsurancePremiumOfThePolicy"])
    
    # Add month and year columns
    df["year"] = df["policyEffectiveDate"].dt.year
    df["month"] = df["policyEffectiveDate"].dt.month
    
    # Merge with discount rate data
    df = df.merge(discount_rates, on=["year", "month"], how="left")
    
    # Calculate years and months difference
    df["yearsDiff"] = CURRENT_YEAR - df["year"]
    df["monthsDiff"] = CURRENT_MONTH - df["month"]
    
    # Compound to present value in a vectorized manner
    df["compoundedValue"] = compound_to_present(
        df["totalInsurancePremiumOfThePolicy"]+df["federalPolicyFee"],
        df["yearsDiff"],
        df["monthsDiff"],
        df["discountRate"]
    )
    
    # Add to running total
    total_compounded_sum += df["compoundedValue"].sum()

print(f"Total Compounded Sum: {total_compounded_sum}")
print(f"Total Annualized Sum: {total_compounded_sum/(2023-1979+1)}")

Processing file: ../Game/State_Policy_Data/State_Policies/propertyState_PR.csv
Processing file: ../Game/State_Policy_Data/State_Policies/propertyState_OK.csv
Processing file: ../Game/State_Policy_Data/State_Policies/propertyState_NY.csv
Processing file: ../Game/State_Policy_Data/State_Policies/propertyState_OR.csv
Processing file: ../Game/State_Policy_Data/State_Policies/propertyState_PA.csv
Processing file: ../Game/State_Policy_Data/State_Policies/propertyState_CA.csv
Processing file: ../Game/State_Policy_Data/State_Policies/propertyState_MP.csv
Processing file: ../Game/State_Policy_Data/State_Policies/propertyState_VI.csv
Processing file: ../Game/State_Policy_Data/State_Policies/propertyState_KS.csv
Processing file: ../Game/State_Policy_Data/State_Policies/propertyState_OH.csv
Processing file: ../Game/State_Policy_Data/State_Policies/propertyState_AL.csv
Processing file: ../Game/State_Policy_Data/State_Policies/propertyState_MA.csv
Processing file: ../Game/State_Policy_Data/State_Pol

In [7]:
# Load and preprocess annual CPI data
cpi_file = "../Local_Data/BLS_Data/US_BLS_CPIAUCSL.csv"  # Replace with the correct path
cpi_data = pd.read_csv(cpi_file, parse_dates=["DATE"])

# Ensure 'CPIAUCSL' is numeric
cpi_data["CPIAUCSL"] = pd.to_numeric(cpi_data["CPIAUCSL"], errors="coerce")

# Handle missing or non-numeric values by forward and backward filling
cpi_data["CPIAUCSL"].fillna(method="ffill", inplace=True)
cpi_data["CPIAUCSL"].fillna(method="bfill", inplace=True)

# Extract year and calculate annual average CPI
cpi_data["year"] = cpi_data["DATE"].dt.year
cpi_data = cpi_data.groupby("year")["CPIAUCSL"].mean().reset_index()
cpi_data = cpi_data.rename(columns={"CPIAUCSL": "CPI"})

# Base CPI for adjustment (e.g., 2024)
BASE_CPI = 313.3

# Running total for CPI-adjusted sum
total_cpi_adjusted_sum = 0

# Process each file
for file in csv_files:
    print(f"Processing file: {file}")
    
    # Load the file and extract required columns
    df = pd.read_csv(
        file, 
        parse_dates=["policyEffectiveDate"], 
        usecols=["policyEffectiveDate", "federalPolicyFee", "totalInsurancePremiumOfThePolicy"]
    )
    
    # Add year column
    df["year"] = df["policyEffectiveDate"].dt.year
    
    # Merge with annual CPI data
    df = df.merge(cpi_data, on="year", how="left")
    
    # Ensure CPI is numeric and filled
    assert df["CPI"].notna().all(), "CPI contains missing values after merge!"
    
    # Calculate inflation-adjusted value
    df["inflationAdjustedValue"] = (
        (df["totalInsurancePremiumOfThePolicy"] + df["federalPolicyFee"]) *
        (BASE_CPI / df["CPI"])  # Adjust for inflation
    )
    
    # Add to running total
    total_cpi_adjusted_sum += df["inflationAdjustedValue"].sum()

# Calculate total annualized sum (adjusting for the number of years)
start_year = cpi_data["year"].min()
end_year = cpi_data["year"].max()
total_years = end_year - start_year + 1
total_annualized_sum = total_cpi_adjusted_sum / total_years

# Output results
print(f"Total CPI-Adjusted Sum: {total_cpi_adjusted_sum}")
print(f"Total Annualized Sum: {total_annualized_sum}")

/tmp/ipykernel_393/2255607804.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  cpi_data["CPIAUCSL"].fillna(method="ffill", inplace=True)
/tmp/ipykernel_393/2255607804.py:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cpi_data["CPIAUCSL"].fillna(method="ffill", inplace=True)
/tmp/ipykernel_393/2255607804.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace meth

Processing file: ../Game/State_Policy_Data/State_Policies/propertyState_PR.csv
Processing file: ../Game/State_Policy_Data/State_Policies/propertyState_OK.csv
Processing file: ../Game/State_Policy_Data/State_Policies/propertyState_NY.csv
Processing file: ../Game/State_Policy_Data/State_Policies/propertyState_OR.csv
Processing file: ../Game/State_Policy_Data/State_Policies/propertyState_PA.csv
Processing file: ../Game/State_Policy_Data/State_Policies/propertyState_CA.csv
Processing file: ../Game/State_Policy_Data/State_Policies/propertyState_MP.csv
Processing file: ../Game/State_Policy_Data/State_Policies/propertyState_VI.csv
Processing file: ../Game/State_Policy_Data/State_Policies/propertyState_KS.csv
Processing file: ../Game/State_Policy_Data/State_Policies/propertyState_OH.csv
Processing file: ../Game/State_Policy_Data/State_Policies/propertyState_AL.csv
Processing file: ../Game/State_Policy_Data/State_Policies/propertyState_MA.csv
Processing file: ../Game/State_Policy_Data/State_Pol

AssertionError: CPI contains missing values after merge!

In [5]:
# Load and preprocess annual CPI data
cpi_file = "../Local_Data/BLS_Data/US_BLS_CPIAUCSL.csv"  # Replace with the correct path
cpi_data = pd.read_csv(cpi_file, parse_dates=["DATE"])

# Step 1: Preprocess CPI data
cpi_data["CPIAUCSL"] = pd.to_numeric(cpi_data["CPIAUCSL"], errors="coerce")
cpi_data["CPIAUCSL"].fillna(method="ffill", inplace=True)
cpi_data["CPIAUCSL"].fillna(method="bfill", inplace=True)
cpi_data["year"] = cpi_data["DATE"].dt.year
cpi_data = cpi_data.groupby("year")["CPIAUCSL"].mean().reset_index()
cpi_data = cpi_data.rename(columns={"CPIAUCSL": "CPI"})

# Base CPI for adjustment (e.g., 2024)
BASE_CPI = 313.3

# Step 2: Function to process each file
def process_file(file_df, cpi_data, base_cpi):
    # Print the file name as it’s being processed
    print(f"Processing file: {file_df}")
    file_df = pd.read_csv(file_df, parse_dates=["policyEffectiveDate"])
    
    # Calculate the present value of premiums for each record
    file_df["year"] = file_df["policyEffectiveDate"].dt.year
    file_df = file_df[["year","totalInsurancePremiumOfThePolicy","federalPolicyFee"]]
    file_grouped = file_df.groupby("year").sum().reset_index()

    # Merge with CPI data
    merged_df = pd.merge(file_grouped, cpi_data, on="year", how="left")
    merged_df["value"] = merged_df["totalInsurancePremiumOfThePolicy"] + merged_df["federalPolicyFee"]

    # Adjust for inflation (assuming 'value' column needs adjustment)
    merged_df["inflation_adjusted_value"] = (
        merged_df["value"] * (base_cpi/ merged_df["CPI"])
    )
    
    # Aggregate adjusted values
    total_adjusted_value = merged_df["inflation_adjusted_value"].sum()
    
    return total_adjusted_value

# Step 3: Iterate over all files
total_inflation_adjusted_sum = 0

for file_df in csv_files:
    val = process_file(file_df, cpi_data, BASE_CPI)
    print(val)
    total_inflation_adjusted_sum += val

# Final result
print("Total Inflation-Adjusted Value:", total_inflation_adjusted_sum)

/tmp/ipykernel_391/4247642833.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  cpi_data["CPIAUCSL"].fillna(method="ffill", inplace=True)
/tmp/ipykernel_391/4247642833.py:7: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cpi_data["CPIAUCSL"].fillna(method="ffill", inplace=True)
/tmp/ipykernel_391/4247642833.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace metho

Processing file: ../Game/State_Policy_Data/State_Policies/propertyState_PR.csv


/tmp/ipykernel_391/4247642833.py:20: DtypeWarning: Columns (3,7,8,10,32,38,39,49,54,57,66,70) have mixed types. Specify dtype option on import or set low_memory=False.
  file_df = pd.read_csv(file_df, parse_dates=["policyEffectiveDate"])


287964323.5166673
Processing file: ../Game/State_Policy_Data/State_Policies/propertyState_OK.csv


/tmp/ipykernel_391/4247642833.py:20: DtypeWarning: Columns (3,7,8,10,32,38,39,49,54,57,66,70) have mixed types. Specify dtype option on import or set low_memory=False.
  file_df = pd.read_csv(file_df, parse_dates=["policyEffectiveDate"])


...

92330352.76713428
Processing file: ../Game/State_Policy_Data/State_Policies/LA/LA_countyCode_22107.0.csv
3795422.6906590494
Processing file: ../Game/State_Policy_Data/State_Policies/LA/LA_countyCode_22113.0.csv


/tmp/ipykernel_391/4247642833.py:20: DtypeWarning: Columns (10,32,38,39,54,66,70) have mixed types. Specify dtype option on import or set low_memory=False.
  file_df = pd.read_csv(file_df, parse_dates=["policyEffectiveDate"])


93681381.0670349
Processing file: ../Game/State_Policy_Data/State_Policies/LA/LA_countyCode_22.0.csv
1852.354529058757
Processing file: ../Game/State_Policy_Data/State_Policies/LA/LA_countyCode_22125.0.csv
1555656.8830378088
Processing file: ../Game/State_Policy_Data/State_Policies/LA/LA_countyCode_22127.0.csv
789829.867267984
Processing file: ../Game/State_Policy_Data/State_Policies/LA/LA_countyCode_22089.0.csv


/tmp/ipykernel_391/4247642833.py:20: DtypeWarning: Columns (10,32,38,39,66,70) have mixed types. Specify dtype option on import or set low_memory=False.
  file_df = pd.read_csv(file_df, parse_dates=["policyEffectiveDate"])


168089209.70476106
Total Inflation-Adjusted Value: 64872409249.17393


In [7]:
# Calculate total annualized sum (adjusting for the number of years)
start_year = cpi_data["year"].min()
end_year = cpi_data["year"].max()
total_years = end_year - start_year + 1
total_annualized_sum = total_inflation_adjusted_sum / total_years

# Output results
print(f"Total CPI-Adjusted Sum: {total_inflation_adjusted_sum}")
print(f"Total Annualized Sum: {total_annualized_sum}")

Total CPI-Adjusted Sum: 64872409249.17393
Total Annualized Sum: 831697554.4765888


In [8]:
# Step 1: Precompute adjustment factors
cpi_data["adjustment_factor"] = BASE_CPI / cpi_data["CPI"]
adjustment_factors = dict(zip(cpi_data["year"], cpi_data["adjustment_factor"]))

# Step 2: Optimized function to process each file
def process_file(file_path, adjustment_factors):
    # Read file
    df = pd.read_csv(file_path, parse_dates=["policyEffectiveDate"])
    
    # Extract year and relevant columns
    df["year"] = df["policyEffectiveDate"].dt.year
    df = df[["year", "totalInsurancePremiumOfThePolicy", "federalPolicyFee"]]
    
    # Group by year and sum
    grouped = df.groupby("year").sum().reset_index()
    
    # Adjust for inflation using precomputed factors
    grouped["adjustment_factor"] = grouped["year"].map(adjustment_factors)
    grouped["value"] = grouped["totalInsurancePremiumOfThePolicy"] + grouped["federalPolicyFee"]
    grouped["inflation_adjusted_value"] = grouped["value"] * grouped["adjustment_factor"]
    
    # Return the total adjusted value
    return grouped["inflation_adjusted_value"].sum()

# Step 3: Process all files
from concurrent.futures import ThreadPoolExecutor

# Use threading for parallel processing
total_inflation_adjusted_sum = 0
with ThreadPoolExecutor() as executor:
    results = executor.map(lambda f: process_file(f, adjustment_factors), csv_files)
    total_inflation_adjusted_sum = sum(results)

# Final result
print("Total Inflation-Adjusted Value:", total_inflation_adjusted_sum)

/tmp/ipykernel_391/4114194550.py:8: DtypeWarning: Columns (38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, parse_dates=["policyEffectiveDate"])
/tmp/ipykernel_391/4114194550.py:8: DtypeWarning: Columns (8,10,32,38,39,54,66,70) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, parse_dates=["policyEffectiveDate"])
/tmp/ipykernel_391/4114194550.py:8: DtypeWarning: Columns (38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, parse_dates=["policyEffectiveDate"])
/tmp/ipykernel_391/4114194550.py:8: DtypeWarning: Columns (38,39,57,66,70) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, parse_dates=["policyEffectiveDate"])
/tmp/ipykernel_391/4114194550.py:8: DtypeWarning: Columns (32,38,39,66,70) have mixed types. Specify dtype option on import or set low_memory=False.


Total Inflation-Adjusted Value: 64872409249.17393


In [9]:
# Calculate total annualized sum (adjusting for the number of years)
start_year = cpi_data["year"].min()
end_year = cpi_data["year"].max()
total_years = end_year - start_year + 1
total_annualized_sum = total_inflation_adjusted_sum / total_years

# Output results
print(f"Total CPI-Adjusted Sum: {total_inflation_adjusted_sum}")
print(f"Total Annualized Sum: {total_annualized_sum}")

Total CPI-Adjusted Sum: 64872409249.17393
Total Annualized Sum: 831697554.4765888
